# codex_prompting.ipynb

Author: Gabriel Grand (grandg@mit.edu) | Date: 01-30-23

Code for prompting codex to generate conditioning statements for physical language tasks.

In [117]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [118]:
import openai
import os
import pandas as pd
import re
import datetime
import pickle
import json

from tqdm import tqdm

In [222]:
# Configure OpenAI API

openai.api_key = 

ENGINE = "code-davinci-002"
MAX_TOKENS = 512
N_SAMPLES = 1
TEMPERATURE = 0.0
STOP = "/**"

In [223]:
RANDOM_SEED = 123

DIR_DATA = "."
DIR_EXPERIMENTS = "../experiments"
DIR_CODEX = "codex"
DIR_COMPLETIONS = "completions"
DIR_PROMPTS = "prompts"

FILE_GENERATIVE_MODEL = os.path.join(DIR_DATA, "prompts.js")
# FILE_STIMULI = os.path.join(DIR_DATA, "phys_lang_examples.csv")
FILE_STIMULI = os.path.join(DIR_DATA, "prompts_grid.csv")

In [224]:
with open(".\\recent_simple_tow.js", "r") as f:
    generative_model_text = f.read()

In [225]:
print(f"{generative_model_text}")

/**
WebPPL code on tug of war world model
 **/

var lazyEffortPrior = 0.25
var normalEffortPrior = 0.5 
var extraEffortPrior = 0.25
var lazyPulling = 0.5
var normalPulling = 1
var extraPulling = 1.5

var strengthSD = 5
var strengthDistribution = Gaussian({mu: 50, sigma: strengthSD})

var tugOfWarModel = function() {

    var strength = mem(function(person) {
        var a = sample(strengthDistribution)
        return a
    })

    var effort = mem(function(person, match) {
        return categorical({
            ps: [lazyEffortPrior, normalEffortPrior, extraEffortPrior],
            vs: [lazyPulling, normalPulling, extraPulling]
        })
    })

    var pulling = function(person, match) {
        return (match == -1) ? categorical({
                ps: [lazyEffortPrior, normalEffortPrior, extraEffortPrior],
                vs: [lazyPulling, normalPulling, extraPulling]
            }) * strength(person) :
            effort(person, match) * strength(person)
    }

    var totalPullin

In [226]:
df = pd.read_csv(".\prompt_grid.csv", index_col="task_id", keep_default_na=False)
df

,language_full,language_phrase_1,language_phrase_2,language_phrase_3,language_phrase_4,question sentence,code_phrase_1,code_phrase_2,code_phrase_3,code_phrase_4,question code
task_id,,,,,,,,,,,
0,"In Game 1, James won against John. In Game 2,...","In Game 1, James won against John.","In Game 2, James won against John.","In Game 3, James won against John.",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['James'],['John'],1) == 1);","condition(winner(['James'],['John'],2) == 1);","condition(winner(['James'],['John'],3) == 1);",,strength(John)
1,"In Game 1, Robert won against Michael. In Gam...","In Game 1, Robert won against Michael.","In Game 2, Michael won against William.","In Game 3, Michael won against David.",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Robert'],['Michael'],1) == 1);","condition(winner(['Michael'],['William'],2) ==...","condition(winner(['Michael'],['David'],3) == 1);",,strength(Michael)
2,"In Game 1, Richard won against Charles. In Ga...","In Game 1, Richard won against Charles.","In Game 2, Joseph won against Charles.","In Game 3, Thomas won against Charles.",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Richard'],['Charles'],1) ==...","condition(winner(['Charles'],['Joseph'],2) == 2);","condition(winner(['Charles'],['Thomas'],3) == 2);",,strength(Charles)
3,"In Game 1, Christopher won against Daniel. In...","In Game 1, Christopher won against Daniel.","In Game 2, Christopher won against Paul.","In Game 3, Christopher won against Mark.",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Christopher'],['Daniel'],1)...","condition(winner(['Christopher'],['Paul'],2) =...","condition(winner(['Christopher'],['Mark'],3) =...",,strength(Daniel)
4,"In Game 1, a team of Donald and George won ag...","In Game 1, a team of Donald and George won ag...","In Game 2, a team of Donald and George won ag...","In Game 3, a team of Donald and George won ag...",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Donald', 'George'],['Kennet...","condition(winner(['Donald', 'George'],['Edward...","condition(winner(['Donald', 'George'],['Ronald...",,strength(George)
5,"In Game 1, a team of Kevin and Jason won agai...","In Game 1, a team of Kevin and Jason won agai...","In Game 2, a team of Kevin and Matthew won ag...","In Game 3, a team of Kevin and Gary won again...",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Kevin', 'Jason'],['Timothy'...","condition(winner(['Kevin', 'Matthew'],['Timoth...","condition(winner(['Kevin', 'Gary'],['Timothy',...",,strength(Jason)
6,"In Game 1, a team of Frank and Scott won agai...","In Game 1, a team of Frank and Scott won agai...","In Game 2, a team of Andrew and Raymond won a...","In Game 3, a team of Andrew and Raymond won a...",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Frank', 'Scott'],['Andrew',...","condition(winner(['Scott', 'Eric'],['Andrew', ...","condition(winner(['Scott', 'Stephen'],['Andrew...",,strength(Scott)
7,"In Game 1, a team of Gregory and Joshua won a...","In Game 1, a team of Gregory and Joshua won a...","In Game 2, a team of Joshua and Jerry won aga...","In Game 3, a team of Joshua and Dennis won ag...",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Gregory', 'Joshua'],['Walte...","condition(winner(['Joshua', 'Jerry'],['Walter'...","condition(winner(['Joshua', 'Dennis'],['Walter...",,strength(Joshua)
8,"In Game 1, a team of Peter and Harold won aga...","In Game 1, a team of Peter and Harold won aga...","In Game 2, a team of Peter and Douglas won ag...","In Game 3, a team of Peter and Henry won agai...",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Peter', 'Harold'],['Carl', ...","condition(winner(['Peter', 'Douglas'],['Ryan',...","condition(winner(['Peter', 'Henry'],['Joe', 'J...",,strength(Harold)


In [227]:
n_phrases_langauge = len(list(filter(lambda col_name: "language_phrase_" in col_name, df.columns)))
n_phrases_code = len(list(filter(lambda col_name: "code_phrase_" in col_name, df.columns)))
assert(n_phrases_langauge == n_phrases_code)
N_PHRASES = n_phrases_langauge
print(N_PHRASES)

4


In [228]:
TEMPLATE_EXAMPLE_HEADER = """
/**
 * Example:
 * {language_full}
 */
"""

def construct_prompt(df, task_id, global_header, n_prompt_examples: int = None):
    prompt = global_header + "\n"
    
    # hold one out
    df_examples = df.drop(df.loc[[task_id]].index)
    
    if n_prompt_examples:
        df_examples = df_examples.sample(n=n_prompt_examples, random_state=RANDOM_SEED)
    
    for _, row in df_examples.iterrows():
        example_header = TEMPLATE_EXAMPLE_HEADER.format(language_full=row["language_full"])
        prompt += example_header
        
        for i in range(1, N_PHRASES + 1):
            if row[f"language_phrase_{i}"]:
                prompt += "\n"
                prompt += "// " + row[f"language_phrase_{i}"] + "\n"
                prompt += row[f"code_phrase_{i}"] + "\n"
                
    prompt += TEMPLATE_EXAMPLE_HEADER.format(language_full=df.loc[[task_id], "language_full"].item())
        
    return prompt, df_examples.index.tolist()

prompt, task_ids = construct_prompt(df, 1, generative_model_text)

In [229]:
print(f"{prompt}")

/**
WebPPL code on tug of war world model
 **/

var lazyEffortPrior = 0.25
var normalEffortPrior = 0.5 
var extraEffortPrior = 0.25
var lazyPulling = 0.5
var normalPulling = 1
var extraPulling = 1.5

var strengthSD = 5
var strengthDistribution = Gaussian({mu: 50, sigma: strengthSD})

var tugOfWarModel = function() {

    var strength = mem(function(person) {
        var a = sample(strengthDistribution)
        return a
    })

    var effort = mem(function(person, match) {
        return categorical({
            ps: [lazyEffortPrior, normalEffortPrior, extraEffortPrior],
            vs: [lazyPulling, normalPulling, extraPulling]
        })
    })

    var pulling = function(person, match) {
        return (match == -1) ? categorical({
                ps: [lazyEffortPrior, normalEffortPrior, extraEffortPrior],
                vs: [lazyPulling, normalPulling, extraPulling]
            }) * strength(person) :
            effort(person, match) * strength(person)
    }

    var totalPullin

In [207]:
completion = openai.Completion.create(
    engine=ENGINE,
    prompt=prompt,
    temperature=TEMPERATURE,
    n=N_SAMPLES,
    stop=STOP,
    max_tokens=MAX_TOKENS,
    logprobs=None,
)

print(completion)

{
  "id": "cmpl-8HtwY9sFk9neo74i5luENFWPjXbbv",
  "object": "text_completion",
  "created": 1699277314,
  "model": "code-davinci-002",
  "choices": [
    {
      "text": "\n//  In Game 1, Robert won against Michael.\ncondition(winner(['Michael'],['Robert'],1) == 2);\n\n//  In Game 2, Michael won against William.\ncondition(winner(['Michael'],['William'],2) == 1);\n\n//  In Game 3, Michael won against David.\ncondition(winner(['Michael'],['David'],3) == 1);\n\n",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 5281,
    "completion_tokens": 95,
    "total_tokens": 5376
  }
}


In [208]:
def query_codex(prompt):
    completion = openai.Completion.create(
        engine=ENGINE,
        prompt=prompt,
        temperature=TEMPERATURE,
        n=N_SAMPLES,
        stop=STOP,
        max_tokens=MAX_TOKENS,
        logprobs=None,
    )
    
    return completion

In [216]:
def extract_conditions(text):
    start, end = "condition", ";"
    print("text",text)
    print("extract condition:   ",[start + x + end for x in re.findall(str(re.escape(start)) + "(.*)" + str(re.escape(end)), text)])
    return [start + x + end for x in re.findall(str(re.escape(start)) + "(.*)" + str(re.escape(end)), text)]

def extract_language(text):
    start, end = "// ", "\n"
    # print("extract lang text2:",re.findall(str(re.escape(start)) + "(.*)" + str(re.escape(end)), text))
    return re.findall(str(re.escape(start)) + "(.*)" + str(re.escape(end)), text)
# //  In Game 1, James won against John.
# cond(winner(['John'],['James'],1) == 2)

# //  In Game 2, James won against John.
# cond(winner(['John'],['James'],2) == 2)

# //  In Game 3, James won against John.
# cond(winner(['John'],['James'],3) == 2)
def parse_choice(choice):
    if choice.finish_reason != "stop":
        print(f"WARNING: Completion choice {choice.index} encountered non-terminal finish reason: {choice.finish_reason}")

    data = {
        "choice_index": choice.index,
        "finish_reason": choice.finish_reason,
        "text": choice.text,
    }
    
    for i, (language, code) in enumerate(zip(extract_language(choice.text), extract_conditions(choice.text))):
        print("lang",language)
        print("code",code)
        data.update({
            f"codex_language_phrase_{i+1}": language,
            f"codex_code_phrase_{i+1}": code,
        })
        
    return data

In [243]:
def run_experiment(df: pd.DataFrame, experiment_id: str = None, restore_ckpt: bool = False, n_prompt_examples: int = 10):
    experiment_id = experiment_id or datetime.datetime.now().strftime('run-%Y-%m-%d-%H-%M-%S')
    ckpt_dir = os.path.join(DIR_EXPERIMENTS, experiment_id, DIR_CODEX)
    os.makedirs(os.path.join(ckpt_dir, DIR_COMPLETIONS), exist_ok=True)
    os.makedirs(os.path.join(ckpt_dir, DIR_PROMPTS), exist_ok=True)
    completions = []
    # Query OpenAI for completions
    for task_id in tqdm(df.index):
        completion_pkl = os.path.join(ckpt_dir, DIR_COMPLETIONS, f"completion_task_{task_id:03d}.pkl")
        # Try loading completion from file
        if restore_ckpt is not None:
            if os.path.exists(completion_pkl):
                with open(completion_pkl, "rb") as f:
                    completions.append(pickle.load(f))
                continue
            else:
                print(f"Completion not found: {completion_pkl}")
            
        prompt_text, prompt_task_ids = construct_prompt(df, task_id, generative_model_text, n_prompt_examples=n_prompt_examples)
        completion = query_codex(prompt_text)
        completions.append(completion)
        with open(completion_pkl, "wb") as f:
            pickle.dump(completion, f)
        with open(os.path.join(ckpt_dir, DIR_PROMPTS, f"prompt_task_{task_id:03d}.json"), "w") as f:
            prompt_json = {
                "task_id": task_id,
                "prompt_task_ids": prompt_task_ids,
                "prompt_text": prompt_text,
            }
            json.dump(prompt_json, f)
    # return completions
    # Parse completions data
    results = []
    for task_id, completion in zip(df.index, completions):
        for choice in completion.choices:
            d = {"task_id": task_id}
            d.update(parse_choice(choice))
            results.append(d)

    results_json = {"results": results}
    
    df_results = pd.DataFrame(results_json["results"]).fillna('')
    df_results = df_results.set_index("task_id")
    
    df_results = df.join(df_results)
    df_results.to_csv(os.path.join(DIR_EXPERIMENTS, RUN_ID, DIR_CODEX, "results1.csv"))
    
    return df_results

In [244]:
# df_results = run_experiment(df)

RUN_ID = "run-2023-11-06"
df_results = run_experiment(df, experiment_id=RUN_ID, restore_ckpt=True)

100%|██████████| 30/30 [00:00<00:00, 1552.99it/s]

text 
//  In Game 1, James won against John.
condition(winner(['John'],['James'],1) == 2);

//  In Game 2, James won against John.
condition(winner(['John'],['James'],2) == 2);

//  In Game 3, James won against John.
condition(winner(['John'],['James'],3) == 2);


extract condition:    ["condition(winner(['John'],['James'],1) == 2);", "condition(winner(['John'],['James'],2) == 2);", "condition(winner(['John'],['James'],3) == 2);"]
lang  In Game 1, James won against John.
code condition(winner(['John'],['James'],1) == 2);
lang  In Game 2, James won against John.
code condition(winner(['John'],['James'],2) == 2);
lang  In Game 3, James won against John.
code condition(winner(['John'],['James'],3) == 2);
text 
//  In Game 1, Robert won against Michael.
condition(winner(['Michael'],['Robert'],1) == 2);

//  In Game 2, Michael won against William.
condition(winner(['Michael'],['William'],2) == 1);

//  In Game 3, Michael won against David.
condition(winner(['Michael'],['David'],3) == 1);




In [251]:
df_results

,language_full,language_phrase_1,language_phrase_2,language_phrase_3,language_phrase_4,question sentence,code_phrase_1,code_phrase_2,code_phrase_3,code_phrase_4,question code,choice_index,finish_reason,text,codex_language_phrase_1,codex_code_phrase_1,codex_language_phrase_2,codex_code_phrase_2,codex_language_phrase_3,codex_code_phrase_3
task_id,,,,,,,,,,,,,,,,,,,,
0,"In Game 1, James won against John. In Game 2,...","In Game 1, James won against John.","In Game 2, James won against John.","In Game 3, James won against John.",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['James'],['John'],1) == 1);","condition(winner(['James'],['John'],2) == 1);","condition(winner(['James'],['John'],3) == 1);",,strength(John),0,stop,"\n// In Game 1, James won against John.\ncond...","In Game 1, James won against John.","condition(winner(['John'],['James'],1) == 2);","In Game 2, James won against John.","condition(winner(['John'],['James'],2) == 2);","In Game 3, James won against John.","condition(winner(['John'],['James'],3) == 2);"
1,"In Game 1, Robert won against Michael. In Gam...","In Game 1, Robert won against Michael.","In Game 2, Michael won against William.","In Game 3, Michael won against David.",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Robert'],['Michael'],1) == 1);","condition(winner(['Michael'],['William'],2) ==...","condition(winner(['Michael'],['David'],3) == 1);",,strength(Michael),0,stop,"\n// In Game 1, Robert won against Michael.\n...","In Game 1, Robert won against Michael.","condition(winner(['Michael'],['Robert'],1) == 2);","In Game 2, Michael won against William.","condition(winner(['Michael'],['William'],2) ==...","In Game 3, Michael won against David.","condition(winner(['Michael'],['David'],3) == 1);"
2,"In Game 1, Richard won against Charles. In Ga...","In Game 1, Richard won against Charles.","In Game 2, Joseph won against Charles.","In Game 3, Thomas won against Charles.",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Richard'],['Charles'],1) ==...","condition(winner(['Charles'],['Joseph'],2) == 2);","condition(winner(['Charles'],['Thomas'],3) == 2);",,strength(Charles),0,stop,"\n// In Game 1, Richard won against Charles.\...","In Game 1, Richard won against Charles.","condition(winner(['Charles'],['Richard'],1) ==...","In Game 2, Joseph won against Charles.","condition(winner(['Charles'],['Joseph'],2) == 2);","In Game 3, Thomas won against Charles.","condition(winner(['Charles'],['Thomas'],3) == 2);"
3,"In Game 1, Christopher won against Daniel. In...","In Game 1, Christopher won against Daniel.","In Game 2, Christopher won against Paul.","In Game 3, Christopher won against Mark.",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Christopher'],['Daniel'],1)...","condition(winner(['Christopher'],['Paul'],2) =...","condition(winner(['Christopher'],['Mark'],3) =...",,strength(Daniel),0,stop,"\n// In Game 1, Christopher won against Danie...","In Game 1, Christopher won against Daniel.","condition(winner(['Daniel'],['Christopher'],1)...","In Game 2, Christopher won against Paul.","condition(winner(['Christopher'],['Paul'],2) =...","In Game 3, Christopher won against Mark.","condition(winner(['Christopher'],['Mark'],3) =..."
4,"In Game 1, a team of Donald and George won ag...","In Game 1, a team of Donald and George won ag...","In Game 2, a team of Donald and George won ag...","In Game 3, a team of Donald and George won ag...",,Question: On a scale from 0 (very weak) to 100...,"condition(winner(['Donald', 'George'],['Kennet...","condition(winner(['Donald', 'George'],['Edward...","condition(winner(['Donald', 'George'],['Ronald...",,strength(George),0,stop,"\n// In Game 1, a team of Donald and George w...","In Game 1, a team of Donald and George won ag...","condition(winner(['Donald', 'George'],['Kennet...","In Game 2, a team of Donald and George won ag...","condition(winner(['Donald', 'George'],['Edward...","In Game 3, a team

In [252]:
df_results.to_csv('file1.csv')

In [254]:
for i in range(1, N_PHRASES + 1):
    df_nonempty_language = df_results[df_results[f"language_phrase_{i}"] != '']
    df_nonempty_code = df_results[df_results[f"code_phrase_{i}"] != '']
    
    agreement_language = (df_nonempty_language[f"language_phrase_{i}"] == df_nonempty_language[f"codex_language_phrase_{i}"]).sum() / len(df_nonempty_language)
    agreement_code = (df_nonempty_code[f"code_phrase_{i}"] == df_nonempty_code[f"codex_code_phrase_{i}"]).sum() / len(df_nonempty_code)
    print(f"language_phrase_{i}: {agreement_language:03f}")
    print(f"code_phrase_{i}: {agreement_code:03f}")

language_phrase_1: 1.000000
code_phrase_1: 0.466667
language_phrase_2: 1.000000
code_phrase_2: 0.538462
language_phrase_3: 1.000000
code_phrase_3: 0.550000


KeyError: 'codex_language_phrase_4'

In [248]:
from IPython.display import display

with pd.option_context('display.max_colwidth', None):
    for i in range(1, N_PHRASES + 1):
        display(df_results[f"code_phrase_{i}"].compare(df_results[f"codex_code_phrase_{i}"], result_names=('truth', 'codex')))

,truth,codex
task_id,,
0,"condition(winner(['James'],['John'],1) == 1);","condition(winner(['John'],['James'],1) == 2);"
1,"condition(winner(['Robert'],['Michael'],1) == 1);","condition(winner(['Michael'],['Robert'],1) == 2);"
2,"condition(winner(['Richard'],['Charles'],1) == 1);","condition(winner(['Charles'],['Richard'],1) == 2);"
3,"condition(winner(['Christopher'],['Daniel'],1) == 1);","condition(winner(['Daniel'],['Christopher'],1) == 2);"
11,"condition(winner(['Keith'],['Samuel'],1) == 2);","condition(winner(['Samuel'],['Keith'],1) == 1);"
14,"condition(winner(['Fred', 'Wayne'],['Billy', 'Steve'],1) == 2);","condition(winner(['Billy', 'Steve'],['Fred', 'Wayne'],1) == 1);"
15,"condition(winner(['Howard', 'Eugene'],['Bobby', 'Victor'],1) == 2);","condition(winner(['Bobby', 'Victor'],['Howard', 'Eugene'],1) == 1);"
16,"condition(winner(['Phillip', 'Todd'],['Alan', 'Shawn'],1) == 2);","condition(winner(['Alan', 'Shawn'],['Phillip', 'Todd'],1) == 1);"
17,"condition(winner(['Clarence', 'Sean'],['Johnny', 'Earl'],1) == 2);","condition(winner(['Johnny', 'Earl'],['Clarence', 'Sean'],1) == 1);"


,truth,codex
task_id,,
0,"condition(winner(['James'],['John'],2) == 1);","condition(winner(['John'],['James'],2) == 2);"
6,"condition(winner(['Scott', 'Eric'],['Andrew', 'Raymond'],2) == 2);","condition(winner(['Andrew', 'Raymond'],['Scott', 'Eric'],2) == 1);"
11,"condition(winner(['Samuel'],['Willie'],2) == 2);","condition(winner(['Willie'],['Samuel'],2) == 1);"
12,"condition(winner(['Nicholas'],['Roy'],2) == 1);","condition(winner(['Roy'],['Nicholas'],2) == 2);"
14,"condition(winner(['Fred', 'Wayne'],['Louis', 'Jeremy'],2) == 2);","condition(winner(['Louis', 'Jeremy'],['Fred', 'Wayne'],2) == 1);"
15,"condition(winner(['Howard', 'Carlos'],['Bobby', 'Martin'],2) == 2);","condition(winner(['Bobby', 'Martin'],['Howard', 'Carlos'],2) == 1);"
17,"condition(winner(['Sean', 'Philip'],['Johnny', 'Earl'],2) == 2);","condition(winner(['Johnny', 'Earl'],['Sean', 'Philip'],2) == 1);"
18,"condition(winner(['Jimmy', 'Danny'],['Mike', 'Stanley'],2) == 2);","condition(winner(['Mike', 'Stanley'],['Jimmy', 'Danny'],2) == 1);"
19,"condition(winner(['Dale', 'Rodney'],['Manuel', 'Curtis'],2) == 2);","condition(winner(['Manuel', 'Curtis'],['Dale', 'Rodney'],2) == 1);"


,truth,codex
task_id,,
0,"condition(winner(['James'],['John'],3) == 1);","condition(winner(['John'],['James'],3) == 2);"
6,"condition(winner(['Scott', 'Stephen'],['Andrew', 'Raymond'],3) == 2);","condition(winner(['Andrew', 'Raymond'],['Scott', 'Stephen'],3) == 1);"
11,"condition(winner(['Samuel'],['Ralph'],3) == 2);","condition(winner(['Ralph'],['Samuel'],3) == 1);"
12,"condition(winner(['Nicholas'],['Benjamin'],3) == 1);","condition(winner(['Benjamin'],['Nicholas'],3) == 2);"
14,"condition(winner(['Fred', 'Wayne'],['Aaron', 'Randy'],3) == 2);","condition(winner(['Aaron', 'Randy'],['Fred', 'Wayne'],3) == 1);"
15,"condition(winner(['Howard', 'Russell'],['Bobby', 'Ernest'],3) == 2);","condition(winner(['Bobby', 'Ernest'],['Howard', 'Russell'],3) == 1);"
17,"condition(winner(['Sean', 'Chris'],['Johnny', 'Earl'],3) == 2);","condition(winner(['Johnny', 'Earl'],['Sean', 'Chris'],3) == 1);"
18,"condition(winner(['Jimmy', 'Bryan'],['Leonard', 'Nathan'],3) == 2);","condition(winner(['Leonard', 'Nathan'],['Jimmy', 'Bryan'],3) == 1);"
19,"condition(winner(['Dale', 'Curtis'],['Manuel', 'Rodney'],3) == 2);","condition(winner(['Manuel', 'Rodney'],['Dale', 'Curtis'],3) == 1);"


KeyError: 'codex_code_phrase_4'

In [249]:
for task_id, row in df_results.iterrows():
    print("---------")
    print(f"task_id {task_id}: {row['language_full']}")
    print()
    
    for i in range(1, N_PHRASES + 1):
        if row[f"language_phrase_{i}"]:
            print(f"Language {i}")
            print(f"TRUTH: {row[f'language_phrase_{i}']}")
            print(f"CODEX: {row[f'codex_language_phrase_{i}']}")
            print()

            print(f"Code {i}")
            print(f"TRUTH: {row[f'code_phrase_{i}']}")
            print(f"CODEX: {row[f'codex_code_phrase_{i}']}")
            print()

---------
task_id 0:  In Game 1, James won against John. In Game 2, James won against John. In Game 3, James won against John.

Language 1
TRUTH:  In Game 1, James won against John.
CODEX:  In Game 1, James won against John.

Code 1
TRUTH: condition(winner(['James'],['John'],1) == 1);
CODEX: condition(winner(['John'],['James'],1) == 2);

Language 2
TRUTH:  In Game 2, James won against John.
CODEX:  In Game 2, James won against John.

Code 2
TRUTH: condition(winner(['James'],['John'],2) == 1);
CODEX: condition(winner(['John'],['James'],2) == 2);

Language 3
TRUTH:  In Game 3, James won against John.
CODEX:  In Game 3, James won against John.

Code 3
TRUTH: condition(winner(['James'],['John'],3) == 1);
CODEX: condition(winner(['John'],['James'],3) == 2);

---------
task_id 1:  In Game 1, Robert won against Michael. In Game 2, Michael won against William. In Game 3, Michael won against David.

Language 1
TRUTH:  In Game 1, Robert won against Michael.
CODEX:  In Game 1, Robert won against 